# Dirty Data Forensics & Visualizing the Void

**Environment:** Google Colab  
**Core Libraries:** `pandas`, `numpy`, `statsmodels`, `missingno`, `seaborn`, `category_encoders`

This notebook is designed for a 50-minute in-class lab. It intentionally includes an **engineered failure** (dummy variable trap) so students can diagnose the algebra behind OLS.

---

## Learning Goals
1. Audit dirty data and identify missingness patterns
2. Use **visual missing-data forensics** (not just `isnull().sum()`)
3. Apply **conditional imputation** under MAR assumptions
4. Trigger and diagnose the **dummy variable trap**
5. Fix multicollinearity and apply **target encoding** for high-cardinality categories


## Step 0: Install any missing packages (Colab only)

If the libraries are already installed, this cell will finish quickly.


In [ ]:
!pip -q install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 2.6 MB/s eta 0:00:00


## Step 1: Environment Initialization and Data Ingestion

### Option A (recommended for class)
Upload `messy_hr_economics.csv` into Colab, then run the load cell below.

### Option B (course GitHub)
If your instructor provides a raw GitHub URL, paste it into `DATA_URL`.


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import category_encoders as ce

pd.set_option("display.max_columns", 100)
sns.set_theme(style="whitegrid")

In [ ]:
# === Data loading ===
# Option A: local file in Colab (after upload)
LOCAL_FILE = "messy_hr_economics.csv"

# Option B: paste a raw GitHub URL here if provided by your instructor
DATA_URL = None  # e.g., "https://raw.githubusercontent.com/your-course-repo/path/messy_hr_economics.csv"

if DATA_URL:
    df = pd.read_csv(DATA_URL)
else:
    df = pd.read_csv(LOCAL_FILE)

print("Shape:", df.shape)
df.head()

Shape: (3200, 13)


,employee_id,department_raw,department,office_zip,tenure_years,age,year_hired,regional_unemployment,regional_cpi_inflation,performance_rating,bonus_pay,base_salary,retention_risk
0,E100000,Sls,Sales,91586,2.3,27,2024,4.30,3.54,3.0,3571.89,64113.01,0.275
1,E100001,ENG.,Engineering,70260,2.2,38,2024,3.52,3.95,4.5,9351.25,113883.39,0.196
2,E100002,Sales,Sales,33731,1.2,44,2025,4.38,0.70,2.5,3046.72,NaN,0.333
3,E100003,SLS.,Sales,62903,3.2,31,2023,6.59,3.10,NaN,NaN,58762.73,0.222
4,E100004,ENG.,Engineering,85512,7.6,40,2019,6.33,2.99,3.5,8669.21,108171.29,0.189


In [ ]:
df.groupby("department")["base_salary"].median()

,base_salary
department,
Engineering,108666.880
Marketing,83365.265
Sales,80097.170


In [ ]:
# Preliminary audit

print("\n--- Descriptive stats (numeric) ---")


print("\n--- Missing values (count) ---")


--- Descriptive stats (numeric) ---

--- Missing values (count) ---


## Step 2: Visual Forensics of Missing Data

**Task:** Plot a missingness matrix using `missingno`.

### Analytical Question
Carefully compare the white gaps for `bonus_pay` and `performance_rating`.

- Do they align horizontally across the same rows?
- If yes, what does that suggest about the missingness mechanism?

> Hint: Perfect row-wise alignment suggests missingness is **structured**, not pure noise.


In [ ]:
# Visual missingness matrix


### Write your diagnosis here (your response)
- **Observed pattern:** ...
- **Likely mechanism (MCAR / MAR / MNAR):** ...
- **Why:** ...


## Step 3: Handling Missingness via Conditional Imputation (MAR)

The `base_salary` column has missing values that **should not** be filled with a global median.

### Task
Impute missing `base_salary` using the **median salary within each department**.

**Syntax hint:**  
`df.groupby(...)[...].transform(lambda x: x.fillna(x.median()))`


In [ ]:
# Inspect missingness in base_salary before imputation


# TODO: Fill base_salary conditionally by department median
# df["base_salary"] = ...

# Your code here

print("Missing base_salary AFTER :", df["base_salary"].isna().sum())

Missing base_salary AFTER : 0


In [ ]:
# Quick check: compare department medians (after imputation)
display(df.groupby("department")["base_salary"].median().to_frame("dept_median_salary"))

## Step 4: Springing the Dummy Variable Trap (Intentional Failure)

You will one-hot encode `department` **without** dropping a baseline category, then add an intercept.  
This creates perfect multicollinearity:

\[
\text{const} = \text{dept_Engineering} + \text{dept_Marketing} + \text{dept_Sales}
\]

### Task
1. Create dummy variables with `pd.get_dummies(...)` (**do not** use `drop_first=True`)
2. Build an OLS design matrix to predict `base_salary`
3. Run the model and **document the warning/failure**


In [ ]:
# Intentional trap
dummies_trap =
X_trap = pd.concat([df[["tenure_years"]], dummies_trap], axis=1)
X_trap = sm.add_constant(X_trap)
y = df["base_salary"]

print("Design matrix columns:", list(X_trap.columns))
display(X_trap.head())

# Try OLS
try:
    model_trap = sm.OLS(y, X_trap).fit()
    print(model_trap.summary())
except Exception as e:
    print("Model failed with exception:")
    print(type(e).__name__, ":", e)

Design matrix columns: ['const', 'tenure_years', 'dept_Engineering', 'dept_Marketing', 'dept_Sales']


,const,tenure_years,dept_Engineering,dept_Marketing,dept_Sales
0,1.0,2.3,False,False,True
1,1.0,2.2,True,False,False
2,1.0,1.2,False,False,True
3,1.0,3.2,False,False,True
4,1.0,7.6,True,False,False


Model failed with exception:
ValueError : Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).


### Socratic Debug Prompt (Important)
Print the first few rows of the constant and dummy columns. For any row:

- What is the sum of `dept_Engineering + dept_Marketing + dept_Sales`?
- What is the value of `const`?

If both are always `1`, then one column is a perfect linear combination of the others.  
That means the matrix is **singular** (determinant = 0), and OLS cannot uniquely solve the coefficients.


In [ ]:
# Algebra check for the dummy variable trap
cols = ["const"] + [c for c in X_trap.columns if c.startswith("dept_")]
check = X_trap[cols].copy()
check["sum_of_dummies"] = check[[c for c in check.columns if c.startswith("dept_")]].sum(axis=1)
display(check.head(10))

print("Are const and sum_of_dummies identical for all rows?",
      np.allclose(check["const"], check["sum_of_dummies"]))

,const,dept_Engineering,dept_Marketing,dept_Sales,sum_of_dummies
0,1.0,False,False,True,1
1,1.0,True,False,False,1
2,1.0,False,False,True,1
3,1.0,False,False,True,1
4,1.0,True,False,False,1
5,1.0,False,False,True,1
6,1.0,False,False,True,1
7,1.0,False,False,True,1
8,1.0,True,False,False,1
9,1.0,False,True,False,1


Are const and sum_of_dummies identical for all rows? True


## Step 5: Escaping the Trap (k-1 Dummies) + Advanced Encoding

### Part A: Fix the dummy variable trap
Recreate department dummies with `drop_first=True`, then rerun OLS.

### Part B: Target encode high-cardinality ZIP codes
`office_zip` has many unique values (high cardinality). Use `category_encoders.TargetEncoder`
to convert it into a single numeric feature representing the average `base_salary` by ZIP.


In [ ]:
# Step 5A: Safe one-hot encoding (k-1 method)
dummies_safe = pd.get_dummies(df["department"], prefix="dept", drop_first=True).astype(int)
X_safe = pd.concat([df[["tenure_years"]], dummies_safe], axis=1)
X_safe = sm.add_constant(X_safe)

model_safe = sm.OLS(y, X_safe).fit()
print(model_safe.summary())

                            OLS Regression Results                            
Dep. Variable:            base_salary   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     3090.
Date:                Fri, 27 Feb 2026   Prob (F-statistic):               0.00
Time:                        10:14:45   Log-Likelihood:                -34010.
No. Observations:                3200   AIC:                         6.803e+04
Df Residuals:                    3196   BIC:                         6.805e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           9.527e+04    372.412    255.

In [ ]:
# Step 5B: Target encoding for high-cardinality ZIP codes
print("Unique ZIP codes:", df["office_zip"].nunique())

encoder = ce.TargetEncoder(cols=["office_zip"])
df["zip_encoded"] = encoder.fit_transform(df[["office_zip"]], df["base_salary"])["office_zip"]

display(df[["office_zip", "zip_encoded", "base_salary"]].head())
print("\nzip_encoded summary:")
display(df["zip_encoded"].describe())

Unique ZIP codes: 874


,office_zip,zip_encoded,base_salary
0,91586,96850.469864,64113.01
1,70260,96921.404305,113883.39
2,33731,92833.861323,80097.17
3,62903,93624.380293,58762.73
4,85512,95127.965538,108171.29



zip_encoded summary:


,zip_encoded
count,3200.000000
mean,94660.067770
std,1802.956702
min,89865.536079
25%,93459.842508
50%,94574.317093
75%,95729.104039
max,101597.080472


## Economic Interpretation (Discussion Prompt)

After running the **safe OLS model**, interpret the coefficients:

- If `Engineering` was dropped (reference category), what does the **intercept** represent?
- What does the coefficient on `dept_Sales` (or `dept_Marketing`) mean?
- Why is target encoding a practical way to incorporate **regional micro-economy effects** without creating 800+ dummy columns?

> Key idea: coefficients on dummy variables are **differences relative to the omitted baseline**.


## Instructor Notes (Hidden/Optional to Discuss Live)

- The alignment of missing `bonus_pay` and `performance_rating` indicates **structured missingness** (commonly diagnosed as **MAR** in this lab context because missingness depends on observed HR process variables like department/tenure).
- `base_salary` missingness is engineered to be MAR by department/tenure, so **conditional median imputation** is more defensible than a global fill.
- The dummy variable trap demonstrates that data transformation is a **linear algebra problem**, not just a coding problem.
